In [ ]:
import pennylane as qml
from pennylane import numpy as np
from qiskit import IBMQ

IBMQ.enable_account(
    "0bd95fc0907cac40148da3abd36d4821f1ad224b12909748bbf88cd4517c2099129991d3f5cac7ca0159907f8c294d48e432bc7d4d8c077ce72abf3dfed19fd6"
)

# Defines the Hamiltonian
H = (
    -6 * qml.Identity(0)
    + 0.5 * qml.PauliZ(0) @ qml.PauliZ(1)
    + 0.5 * qml.PauliZ(1) @ qml.PauliZ(2)
    + 0.5 * qml.PauliZ(2) @ qml.PauliZ(3)
    + 0.5 * qml.PauliZ(3) @ qml.PauliZ(4)
    + 0.5 * qml.PauliZ(4) @ qml.PauliZ(5)
)

# Creates the IBMQ device
dev = qml.device("qiskit.ibmq", wires=6, backend="ibmq_quito", shots=1024)

@qml.qnode(dev)
def circuit(params):
    for p, w in zip(params, range(6)):
        qml.RY(p, wires=w)
    return qml.expval(H)

@qml.qnode(dev)
def sample_circuit(params):
    for p, w in zip(params, range(6)):
        qml.RY(p, wires=w)
    return qml.sample()

# 1) Initial expectation
init = circuit([0]*6)
print("Initial Energy:", init)

# 2) Optimisation
opt = qml.AdagradOptimizer(stepsize=0.5)
params = np.random.rand(6)
for _ in range(200):
    params = opt.step(circuit, params)

# 3) Final energy & sampling
final = circuit(params)
print("Final Energy: ", final)
print("Sampled Bitstring:", sample_circuit(params))
